## References
https://docs.opencv.org/4.5.2/da/d53/tutorial_py_houghcircles.html

https://theailearner.com/tag/hough-gradient-method-opencv/

In [4]:
import os
import cv2
import numpy as np


Hue is define from 0 to 360 degrees as follows where blue is at 240 degrees and ranges from about 210 to 270

<img src ='hue _image.png'/>

However, in Python/OpenCV, hues are scaled to the range 0 to 180. So all hues from the chart are divided by 2.


 ####           The HSV ranges for red green and yellow lights are taken from the below HSV chart
<img src = hue_scale.jpg/>

https://i.stack.imgur.com/TSKh8.png

In [193]:
def detect_signal(filepath: str, file: str, count: int):
    '''Detects individual traffic lights, masks them, highlights the signal by drawing cricles around it 
    and displays the state of the traffic signal
    
    Args: 
    filepath (str) : Destination path of images stored
    file (str) : files in the path folder
    count (int) : Counter
    
    '''

    font = cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.imread(filepath+ "\\" + file)
    cimg = img
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Hue color range for red
    # lower mask (0-10)
    # red can have Hue value in range, let's say from 0 to 10, as well as in range from 170 to 180.
    lower_red1 = np.array([0,100,100])
    upper_red1 = np.array([10,255,255])
    
    # upper mask (160-180) 
    lower_red2 = np.array([160,65,65])
    upper_red2 = np.array([180,255,255])
    
    # Hue color range for green
    lower_green = np.array([35,80,80])
    upper_green = np.array([95,255,255])
    
    # Hue color range for green
    lower_yellow = np.array([20,120,120])
    upper_yellow = np.array([35,255,255])
    
    # inRange: masks and focus/highlights only on the portion of the image with specified lower and upper HSV ranges
    # since red has two ranges in hsv we need to add them
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.add(mask1, mask2)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
   
    # print size of image
    size = img.shape

    # detects regions of specific color in the masked image through hough circle transform using hough gradient method 
    #return vector of circles with their coordinates
    r_circles = cv2.HoughCircles(mask_red, cv2.HOUGH_GRADIENT, 1, 50,
                               param1=50, param2=10, minRadius=0, maxRadius=30)

    g_circles = cv2.HoughCircles(mask_green, cv2.HOUGH_GRADIENT, 1, 60,
                                 param1=50, param2=10, minRadius=0, maxRadius=30)

    y_circles = cv2.HoughCircles(mask_yellow, cv2.HOUGH_GRADIENT, 1, 30,
                                 param1=50, param2=5, minRadius=0, maxRadius=30)

    # range to detect the circles in the masked image. if r is too high no circles are detected.
    r = 5
    # limits the detection only to traffic lights.
    #If not specified detects everything in specified color range in the image 
    bound = 4.0 / 10
    
    #check whether circles are detected or not
    if r_circles is not None:
        #returns (X,Y, r) center co-ordinates and radius r
        r_circles = np.uint16(np.around(r_circles))
        #Loops through all the circles
        #Here i[0] = X coordinate of circle center; i[1] = Y coordinate of circle center; i[2] = r Radius
        for i in r_circles[0, :]:
            # ignoring or skipping the circles with are out of the image dimensions
            if i[0] > size[1] or i[1] > size[0]or i[1] > size[0]*bound:
                continue
            # intializing h and s to determine the range as to when circles to be drawn
            # this ignores small unwanted masked regions
            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):
                    #ignores regions which are not circles
                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += mask_red[i[1]+m, i[0]+n]
                    s += 1
            
            # range upon when circles to be drawn
            # this ignores small unwanted masked regions
            if h / s > 50:
                #draws circles at (X,Y,r)
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(mask_red, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'RED',(i[0], i[1]), font, 0.8,(255,0,0),2,cv2.LINE_AA)

    if g_circles is not None:
        g_circles = np.uint16(np.around(g_circles))

        for i in g_circles[0, :]:
            if i[0] > size[1] or i[1] > size[0] or i[1] > size[0]*bound:
                continue

            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):

                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += mask_green[i[1]+m, i[0]+n]
                    s += 1
            if h / s > 100:
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(mask_green, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'GREEN',(i[0], i[1]), font, 1,(255,0,0),2,cv2.LINE_AA)

    if y_circles is not None:
        y_circles = np.uint16(np.around(y_circles))

        for i in y_circles[0, :]:
            if i[0] > size[1] or i[1] > size[0] or i[1] > size[0]*bound:
                continue

            h, s = 0.0, 0.0
            for m in range(-r, r):
                for n in range(-r, r):

                    if (i[1]+m) >= size[0] or (i[0]+n) >= size[1]:
                        continue
                    h += mask_yellow[i[1]+m, i[0]+n]
                    s += 1
            if h / s > 50:
                cv2.circle(cimg, (i[0], i[1]), i[2]+10, (0, 255, 0), 2)
                cv2.circle(mask_yellow, (i[0], i[1]), i[2]+30, (255, 255, 255), 2)
                cv2.putText(cimg,'YELLOW',(i[0], i[1]), font, 0.8,(255,0,0),2,cv2.LINE_AA)

    cv2.imshow('Traffic_labels' + str(count), cimg)
    #writes image to the specified path
    path = r"C:\Users\rajiv\Practise\Traffic Light Detection - OpenCV\Traffic_labels\Output"
    cv2.imwrite(os.path.join(path, 'traffic_labels' + str(count) + '.jpg'), cimg)
    
    #masked images for reference
#     cv2.imshow('mask_red', mask_red)
#     cv2.imshow('mask_green', mask_green)
#     cv2.imshow('mask_yellow', mask_yellow)

    # waits until a key is pressed
    cv2.waitKey(0)
    # destroys the window showing image after any key is pressed
    cv2.destroyAllWindows()

In [195]:
if __name__ == '__main__':
    
    count = 0
    path = r"C:\Users\rajiv\Practise\Traffic Light Detection - OpenCV\Traffic_labels\Images"
    for f in os.listdir(path):
        if f.endswith('.JPG') or f.endswith('.jpg'):
            count +=1
            detect_signal(path, f, count)